### Load Images

In [1]:
import tensorflow as tf
from glob import glob
import numpy as np
import cv2

In [2]:
# function to take filenames and convert to numpy arrays
# TODO: use pickle to save the dataset instead
def convert_images(files):
    return np.asarray([cv2.imread(file) for file in files])

In [2]:
filenames = glob("./train/*")
filenames = sorted(filenames, key = lambda x:int(x.split('\\')[1].split('.png')[0]))

In [ ]:
image_data = convert_images(filenames)

In [5]:
image_data.shape

(50000, 32, 32, 3)

In [4]:
#np.save('train.npy',image_data)

In [3]:
image_data = np.load('train.npy')

### Load Labels

In [4]:
import pandas as pd

In [5]:
labels_df = pd.read_csv('trainLabels.csv')

In [6]:
labels_unique = labels_df.label.unique()

In [7]:
labels_unique

array(['frog', 'truck', 'deer', 'automobile', 'bird', 'horse', 'ship',
       'cat', 'dog', 'airplane'], dtype=object)

In [8]:
# just get the id based on the filename
train_indeces = list((int(filenames[i].split('\\')[1].split('.png')[0]) for i in range(len(filenames))))

In [9]:
# get real labels of each
train_labels = [labels_df['label'][idx-1] for idx in train_indeces]

In [10]:
# convert each real label to corresponding index (0-9)
label_data = [np.where(label==labels_unique)[0][0] for label in train_labels]

### Split Training/Validation

In [11]:
from sklearn.model_selection import train_test_split

image_train, image_val, label_train, label_val = train_test_split(image_data,label_data,test_size=0.1,random_state = 7)
image_val.shape

(5000, 32, 32, 3)

## Training

For this project, we'll be utilizing [convolutional neural networks](https://en.wikipedia.org/wiki/Convolutional_neural_network). They are excellent for image problems due to their ability to capture spatial information of the pixels. A good summary can be found [here](http://cs231n.github.io/convolutional-networks/)!

Specifically, we implemented [Residual Networks](https://arxiv.org/abs/1512.03385), a popular architecture that uses a novel method for constructing a network. 

The original CNN architectures were found to have worse performance as the number of layers increase - it became increasingly difficult to backpropagate through a huge amount of layers. 

Residual Networks (ResNet) aims to fix this by introducing a "residual block" (seen below)

<img src='./resblock.png'>

The aim of these residual blocks is for the network to **learn whether or not convolutions in a layer are useful or not**. To do this, the output of a block will always include the identity of the inputs. 

If the identity transformation fit the data the most, it would be easier to zero out the weights of the convolutions in this block, rather than fitting the convolutions to an identity mapping.

Mathematically, let $F(x)$ be the two convolutional layers within the block. Thus our residual function $H(x)$ will just be:

$$ H(x) = F(x) + x $$

This can be easily backpropagated with since:

$$ H'(x) = F'(x) + I $$

Where $I$ is the $n x n$ identity matrix, given that $x$ as $n$ inputs.

### Create Graph

To actually implement this, we used [Tensorflow](https://www.tensorflow.org/).

A summary of the process can be described in these steps:

- Data Augmentation
- Mean Centering Images (take mean of R,G,B channels)
- Architecture of Actual Network:
    - 1 Convolutional Layer
        - kernel_size = (3,3)
        - stride = 1
        - num_outputs = 16
        - Batch Norm then passed into Relu Activation
    - 3 Residual Block Groups
        - n residual blocks
            - kernel_size = (3,3)
            - stride = 1
            - num_outputs = [16,32,64] (16 for first group, 32 for second, 64 for last)
    - 1 Global Average Pooling Layer
        - Avg Pooling, No padding
        - kernel_size = (8,8)
    - 1 Fully Connected Layer
        - num_outputs = 10
- Loss: Cross entropy + WEIGHT_DECAY * (L2 Weight Regularization)
- Optimizer: SGD with Momentum of 0.9
    - Initial learning rate of 0.1
    - Decay by 1/10 at iteration 32,000 and 48,000


** Data Augmentation **
- This is an important step because it allows us to "artificially" increase our training set.
- More data = more information the neural network can learn
- Helps increase generalization
- Doing two augmentations:
    - Pad around image by 4 pixels and randomly crop a new 32x32x3 image
    - Random horizontal flip
    
**Changing Dimension between Residual Blocks** 
- There will be 3 Residual Blocks in the network
- When connecting one to the next group, output channels will not match.
    - e.g. First block group outputs 16 channels, second group outputs 32 channels. Cannot add $F(x) + x$ in first residual of second group, since $x$ has 16 channels whereas $F(x)$ will have 32.
- Solution is to modify x so that the dimensions match:
    - (A) You can pad 0 channels on x to match (We used this)
    - (B) Use a 1x1 Convolution with desired output channels as a projection layer
- Of course, we want to reduce the Width/Height dimensions going from group to group, so we also add a stride of 2 to any operation
    - (A) Avg-pool with stride 2 before padding 0 channels
    - (B) Just do the convolution with a stride of 2


#### Resnet Size
- This can be defined by a parameter $n$, which denotes how many residual blocks per group
- We tried Resnet-20 and Resnet-32 (n=3, n=5), but can be done for more with more compute power

### Tensorflow Implementation

In [12]:
red_mean = np.mean(image_train[:,:,:,0])
green_mean = np.mean(image_train[:,:,:,1])
blue_mean = np.mean(image_train[:,:,:,2])

In [13]:
def residual(inputs = None, num_outputs = 16,is_training = False,projection = False, weight_decay = 1e-4):
    identity = inputs # keep output of last layer
    inputs = tf.nn.relu(tf.contrib.layers.batch_norm(inputs = inputs,is_training = is_training)) # make sure to pass through activation
    if projection:
        identity = tf.contrib.layers.avg_pool2d(inputs = identity, stride = 2, kernel_size = (2,2))
        identity = tf.pad(identity, [[0,0],[0,0],[0,0],[num_outputs//4,num_outputs//4]]) # pad with 0s to keep # filters the same
    conv1 = tf.contrib.layers.conv2d(inputs = inputs, num_outputs = num_outputs, stride = 2 if projection else 1, kernel_size = (3,3), activation_fn = None, weights_regularizer = tf.contrib.layers.l2_regularizer(weight_decay))
    bn1 = tf.nn.relu(tf.contrib.layers.batch_norm(inputs = conv1, is_training = is_training))
    conv2 = tf.contrib.layers.conv2d(inputs = bn1, num_outputs = num_outputs, stride = 1, kernel_size = (3,3), activation_fn = None, weights_regularizer = tf.contrib.layers.l2_regularizer(weight_decay))
    return tf.add(identity,conv2)

In [14]:
tf.reset_default_graph()
weight_decay = 1e-4

inputs = tf.placeholder(tf.float32, (None,32,32,3))
labels = tf.placeholder(tf.int64, (None), name='labels')
learning_rate = tf.placeholder(tf.float32, (None))
is_training = tf.placeholder(tf.bool, (None))

def image_augment(I):
    # pad 4 zeros in width and height
    I = tf.pad(I,[[4,4],[4,4],[0,0]])
    # crop new 32 x 32 image from padded image
    I = tf.random_crop(I,[32,32,3])
    # flip left/right
    I = tf.image.random_flip_left_right(I)
    return I

def normalize(I, means):
    # isolate color channels and make sure rank 3 tensors
    new_red = tf.expand_dims((I[:,:,0]-means[0])/255,-1)
    new_blue = tf.expand_dims((I[:,:,1]-means[1])/255,-1)
    new_green = tf.expand_dims((I[:,:,2]-means[2])/255,-1)
    
    return tf.concat([new_red,new_blue,new_green], axis = 2)


augmented = tf.map_fn(image_augment,inputs)
final_input = tf.identity(augmented)
whitened = tf.map_fn(lambda I: normalize(I,[red_mean,green_mean,blue_mean]), final_input)

n_outputs = [16,32,64]
n = 5 # 3 -> 20 layers, 5-> 32 layers, 7->44 layers

layer1 = tf.contrib.layers.conv2d(inputs = whitened, num_outputs = 16, stride = 1, kernel_size = (3,3), activation_fn = None, weights_regularizer = tf.contrib.layers.l2_regularizer(weight_decay))
res_layer = residual(inputs = layer1,num_outputs = 16,is_training = is_training)

for i,outputs in enumerate(n_outputs):
    for j in range(n-1):
        res_layer = residual(inputs = res_layer, num_outputs = outputs, is_training = is_training)
    if (i<2):
        res_layer = residual(inputs = res_layer, num_outputs = n_outputs[i+1],is_training=is_training,projection=True)
        
res_layer = tf.nn.relu(tf.contrib.layers.batch_norm(inputs=res_layer,is_training=is_training))
global_avg_pool = tf.contrib.layers.avg_pool2d(inputs=res_layer,stride = 1, kernel_size = (8,8),padding='VALID')

fc = tf.contrib.layers.fully_connected(inputs=tf.contrib.layers.flatten(global_avg_pool),num_outputs = 10,weights_regularizer=tf.contrib.layers.l2_regularizer(weight_decay),activation_fn=None)
print(fc.shape)
output = tf.identity(fc, name='output')
    
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=output, labels=labels)) + tf.losses.get_regularization_loss()
optimizer = tf.train.MomentumOptimizer(learning_rate,0.9)

with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
    minimizer = optimizer.minimize(loss)

correct = tf.equal(tf.argmax(output, 1), labels)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

# Source: CS342 with Dr. Krähenbühl
print( "Total number of variables used:", np.sum([v.get_shape().num_elements() for v in tf.trainable_variables()]))

(?, 10)
Total number of variables used: 464154


### Training for 200 Epochs

- Epochs are just iterations
- Each iteration we shuffle the dataset and then feed in the images into the CNN in batches (GPU can't fit all images at once).

### Resnet 32

In [15]:
# Batch size
BS = 128

# Start a session
sess = tf.Session()

# Set up training
sess.run(tf.global_variables_initializer())
steps = [32000,48000]
it = 0
lr = 0.1
epoch = 0
last_acc = 0
first_div = False
while it < 64000:
    # Let's shuffle the data every epoch
    len_ = image_train.shape[0]
    np.random.seed(epoch)
    np.random.shuffle(image_train)
    np.random.seed(epoch)
    np.random.shuffle(label_train)
    # Go through the entire dataset once
    accuracy_train, loss_train = [], []
    final = 0
    for i in range(0, image_train.shape[0]-BS+1, BS):
        if it >= 64000:
            break
        if it in steps:
            lr /= 10
            first_div = True
        # Train a single batch
        batch_images, batch_labels = image_train[i:i+BS], label_train[i:i+BS]
        final = i+BS #useful for training last batch
        accuracy_, loss_, _ = sess.run([accuracy, loss, minimizer], feed_dict={inputs: batch_images, labels: batch_labels, is_training:True,learning_rate:lr})
        accuracy_train.append(accuracy_)
        loss_train.append(loss_)
        it+=1
    # train final batch
    if it >= 64000:
        break
    if it in steps:
        lr /= 10
        first_div = True
    batch_images, batch_labels = image_data[final:len_], label_data[final:len_]
    accuracy_, loss_, _ = sess.run([accuracy, loss, minimizer], feed_dict={inputs: batch_images, labels: batch_labels, is_training:True,learning_rate:lr})
    accuracy_train.append(accuracy_)
    loss_train.append(loss_)
    it+=1
    
    # compute validation results
    len_val = image_val.shape[0]
    accuracy_val, loss_val = [], []
    final = 0
    for i in range(0,len_val-BS+1,BS):
        batch_images, batch_labels = image_val[i:i+BS], label_val[i:i+BS]
        final = i+BS
        accuracy_, loss_ = sess.run([accuracy,loss],feed_dict={final_input:batch_images,labels:batch_labels,is_training:False})
        accuracy_val.append(accuracy_)
        loss_val.append(loss_)
    # test final batch
    batch_images, batch_labels = image_val[final:len_val], label_val[final:len_val]
    accuracy_, loss_ = sess.run([accuracy,loss],feed_dict={final_input:batch_images,labels:batch_labels,is_training:False})
    accuracy_val.append(accuracy_)
    loss_val.append(loss_)
    
    last_acc = np.mean(accuracy_train)
    print('[%3d] Train Accuracy: %0.3f  \t  Train Loss: %0.3f \t Val. Accuracy %0.3f \t Val. Loss %.3f '%(epoch, last_acc, np.mean(loss_train),np.mean(accuracy_val),np.mean(loss_val)))
    epoch += 1

print('[%3d] Train Accuracy: %0.3f  \t  Train Loss: %0.3f \t Val. Accuracy %0.3f \t Val. Loss %.3f '%(epoch, last_acc, np.mean(loss_train),np.mean(accuracy_val),np.mean(loss_val)))

[  0] Train Accuracy: 0.410  	  Train Loss: 1.653 	 Val. Accuracy 0.159 	 Val. Loss 7.113 
[  1] Train Accuracy: 0.608  	  Train Loss: 1.153 	 Val. Accuracy 0.187 	 Val. Loss 24.514 
[  2] Train Accuracy: 0.711  	  Train Loss: 0.905 	 Val. Accuracy 0.291 	 Val. Loss 7.490 
[  3] Train Accuracy: 0.757  	  Train Loss: 0.787 	 Val. Accuracy 0.330 	 Val. Loss 6.315 
[  4] Train Accuracy: 0.787  	  Train Loss: 0.710 	 Val. Accuracy 0.365 	 Val. Loss 5.644 
[  5] Train Accuracy: 0.807  	  Train Loss: 0.660 	 Val. Accuracy 0.567 	 Val. Loss 1.826 
[  6] Train Accuracy: 0.820  	  Train Loss: 0.623 	 Val. Accuracy 0.535 	 Val. Loss 2.258 
[  7] Train Accuracy: 0.832  	  Train Loss: 0.594 	 Val. Accuracy 0.748 	 Val. Loss 0.983 
[  8] Train Accuracy: 0.844  	  Train Loss: 0.572 	 Val. Accuracy 0.690 	 Val. Loss 1.079 
[  9] Train Accuracy: 0.851  	  Train Loss: 0.549 	 Val. Accuracy 0.665 	 Val. Loss 1.242 
[ 10] Train Accuracy: 0.858  	  Train Loss: 0.536 	 Val. Accuracy 0.717 	 Val. Loss 1.038

[ 91] Train Accuracy: 0.974  	  Train Loss: 0.283 	 Val. Accuracy 0.891 	 Val. Loss 0.575 
[ 92] Train Accuracy: 0.981  	  Train Loss: 0.261 	 Val. Accuracy 0.906 	 Val. Loss 0.537 
[ 93] Train Accuracy: 0.983  	  Train Loss: 0.253 	 Val. Accuracy 0.908 	 Val. Loss 0.526 
[ 94] Train Accuracy: 0.985  	  Train Loss: 0.247 	 Val. Accuracy 0.910 	 Val. Loss 0.525 
[ 95] Train Accuracy: 0.986  	  Train Loss: 0.241 	 Val. Accuracy 0.908 	 Val. Loss 0.538 
[ 96] Train Accuracy: 0.989  	  Train Loss: 0.235 	 Val. Accuracy 0.907 	 Val. Loss 0.550 
[ 97] Train Accuracy: 0.989  	  Train Loss: 0.233 	 Val. Accuracy 0.910 	 Val. Loss 0.534 
[ 98] Train Accuracy: 0.991  	  Train Loss: 0.228 	 Val. Accuracy 0.914 	 Val. Loss 0.527 
[ 99] Train Accuracy: 0.991  	  Train Loss: 0.225 	 Val. Accuracy 0.913 	 Val. Loss 0.544 
[100] Train Accuracy: 0.991  	  Train Loss: 0.222 	 Val. Accuracy 0.914 	 Val. Loss 0.534 
[101] Train Accuracy: 0.992  	  Train Loss: 0.220 	 Val. Accuracy 0.909 	 Val. Loss 0.546 

### Resnet 20

In [15]:
# Batch size
BS = 128

# Start a session
sess = tf.Session()

# Set up training
sess.run(tf.global_variables_initializer())
steps = [32000,48000]
it = 0
lr = 0.1
epoch = 0
last_acc = 0
first_div = False
while it < 64000:
    # Let's shuffle the data every epoch
    len_ = image_train.shape[0]
    np.random.seed(epoch)
    np.random.shuffle(image_train)
    np.random.seed(epoch)
    np.random.shuffle(label_train)
    # Go through the entire dataset once
    accuracy_train, loss_train = [], []
    final = 0
    for i in range(0, image_train.shape[0]-BS+1, BS):
        if it >= 64000:
            break
        if it in steps:
            lr /= 10
            first_div = True
        # Train a single batch
        batch_images, batch_labels = image_train[i:i+BS], label_train[i:i+BS]
        final = i+BS #useful for training last batch
        accuracy_, loss_, _ = sess.run([accuracy, loss, minimizer], feed_dict={inputs: batch_images, labels: batch_labels, is_training:True,learning_rate:lr})
        accuracy_train.append(accuracy_)
        loss_train.append(loss_)
        it+=1
    # train final batch
    if it >= 64000:
        break
    if it in steps:
        lr /= 10
        first_div = True
    batch_images, batch_labels = image_data[final:len_], label_data[final:len_]
    accuracy_, loss_, _ = sess.run([accuracy, loss, minimizer], feed_dict={inputs: batch_images, labels: batch_labels, is_training:True,learning_rate:lr})
    accuracy_train.append(accuracy_)
    loss_train.append(loss_)
    it+=1
    
    # compute validation results
    len_val = image_val.shape[0]
    accuracy_val, loss_val = [], []
    final = 0
    for i in range(0,len_val-BS+1,BS):
        batch_images, batch_labels = image_val[i:i+BS], label_val[i:i+BS]
        final = i+BS
        accuracy_, loss_ = sess.run([accuracy,loss],feed_dict={final_input:batch_images,labels:batch_labels,is_training:False})
        accuracy_val.append(accuracy_)
        loss_val.append(loss_)
    # test final batch
    batch_images, batch_labels = image_val[final:len_val], label_val[final:len_val]
    accuracy_, loss_ = sess.run([accuracy,loss],feed_dict={final_input:batch_images,labels:batch_labels,is_training:False})
    accuracy_val.append(accuracy_)
    loss_val.append(loss_)
    
    last_acc = np.mean(accuracy_train)
    print('[%3d] Train Accuracy: %0.3f  \t  Train Loss: %0.3f \t Val. Accuracy %0.3f \t Val. Loss %.3f '%(epoch, last_acc, np.mean(loss_train),np.mean(accuracy_val),np.mean(loss_val)))
    epoch += 1

print('[%3d] Train Accuracy: %0.3f  \t  Train Loss: %0.3f \t Val. Accuracy %0.3f \t Val. Loss %.3f '%(epoch, last_acc, np.mean(loss_train),np.mean(accuracy_val),np.mean(loss_val)))

[  0] Train Accuracy: 0.431  	  Train Loss: 1.568 	 Val. Accuracy 0.136 	 Val. Loss 5.716 
[  1] Train Accuracy: 0.620  	  Train Loss: 1.111 	 Val. Accuracy 0.181 	 Val. Loss 18.423 
[  2] Train Accuracy: 0.701  	  Train Loss: 0.910 	 Val. Accuracy 0.433 	 Val. Loss 4.424 
[  3] Train Accuracy: 0.745  	  Train Loss: 0.800 	 Val. Accuracy 0.462 	 Val. Loss 4.034 
[  4] Train Accuracy: 0.773  	  Train Loss: 0.724 	 Val. Accuracy 0.591 	 Val. Loss 2.656 
[  5] Train Accuracy: 0.795  	  Train Loss: 0.676 	 Val. Accuracy 0.640 	 Val. Loss 1.278 
[  6] Train Accuracy: 0.809  	  Train Loss: 0.642 	 Val. Accuracy 0.474 	 Val. Loss 2.757 
[  7] Train Accuracy: 0.821  	  Train Loss: 0.609 	 Val. Accuracy 0.634 	 Val. Loss 1.387 
[  8] Train Accuracy: 0.830  	  Train Loss: 0.587 	 Val. Accuracy 0.575 	 Val. Loss 1.974 
[  9] Train Accuracy: 0.838  	  Train Loss: 0.568 	 Val. Accuracy 0.596 	 Val. Loss 1.514 
[ 10] Train Accuracy: 0.845  	  Train Loss: 0.551 	 Val. Accuracy 0.696 	 Val. Loss 1.116

[ 91] Train Accuracy: 0.959  	  Train Loss: 0.308 	 Val. Accuracy 0.892 	 Val. Loss 0.533 
[ 92] Train Accuracy: 0.966  	  Train Loss: 0.287 	 Val. Accuracy 0.897 	 Val. Loss 0.512 
[ 93] Train Accuracy: 0.971  	  Train Loss: 0.274 	 Val. Accuracy 0.899 	 Val. Loss 0.528 
[ 94] Train Accuracy: 0.972  	  Train Loss: 0.267 	 Val. Accuracy 0.902 	 Val. Loss 0.500 
[ 95] Train Accuracy: 0.976  	  Train Loss: 0.258 	 Val. Accuracy 0.906 	 Val. Loss 0.495 
[ 96] Train Accuracy: 0.976  	  Train Loss: 0.255 	 Val. Accuracy 0.904 	 Val. Loss 0.506 
[ 97] Train Accuracy: 0.978  	  Train Loss: 0.250 	 Val. Accuracy 0.904 	 Val. Loss 0.515 
[ 98] Train Accuracy: 0.978  	  Train Loss: 0.245 	 Val. Accuracy 0.905 	 Val. Loss 0.495 
[ 99] Train Accuracy: 0.980  	  Train Loss: 0.242 	 Val. Accuracy 0.901 	 Val. Loss 0.536 
[100] Train Accuracy: 0.980  	  Train Loss: 0.238 	 Val. Accuracy 0.906 	 Val. Loss 0.501 
[101] Train Accuracy: 0.980  	  Train Loss: 0.237 	 Val. Accuracy 0.908 	 Val. Loss 0.500 

### Save Model

In [16]:
# Source: CS342 with Dr. Krähenbühl
def save(output_file, graph=None, session=None):
    from zipfile import ZipFile
    from os import path, remove, rmdir
    from tempfile import mkdtemp

    tmp_dir = mkdtemp()
    tmp_output = path.join(tmp_dir, path.basename(output_file))
    with graph.as_default():
        saver = tf.train.Saver(allow_empty=True)
        saver.save(session, tmp_output, write_state=False)

    of = ZipFile(output_file, 'w')
    for f in glob(tmp_output+'.*'):
        of.write(f, path.basename(f))
        remove(f)
    of.close()
    rmdir(tmp_dir)

In [17]:
graph = tf.get_default_graph()
save('./resnet32.tfg',graph,sess)

### Load Model





In [31]:
def load(input_file, graph=None, session=None):
    from os import path
    from shutil import rmtree
    from tempfile import mkdtemp
    from zipfile import ZipFile

    tmp_dir = mkdtemp()

    f = ZipFile(input_file, 'r')
    f.extractall(tmp_dir)
    f.close()

    # Find the model name
    meta_files = glob(path.join(tmp_dir, '*.meta'))
    if len(meta_files) < 1:
        raise IOError( "[E] No meta file found, giving up" )
    if len(meta_files) > 1:
        raise IOError( "[E] More than one meta file found, giving up" )

    meta_file = meta_files[0]
    model_file = meta_file.replace('.meta', '')

    if graph is None:
        graph = tf.get_default_graph()
    if session is None:
        session = tf.get_default_session()
    if session is None:
        session = tf.Session()

    # Load the model in TF
    with graph.as_default():
        saver = tf.train.import_meta_graph(meta_file)
        if saver is not None:
            saver.restore(session, model_file)
    rmtree(tmp_dir)
    return graph

In [ ]:
graph = load('./week4_model_temp.tfg')

### Make Test Predictions

In [8]:
test_files = glob('./test/*')
test_files = sorted(test_files, key = lambda x:int(x.split('\\')[1].split('.png')[0]))

In [11]:
image_test = convert_images(test_files)

In [12]:
np.save('test.npy',image_test)

In [46]:
image_test.shape

(300000, 32, 32, 3)

In [ ]:
sess = tf.Session()
with graph.as_default():
    test_batch = 3000 # batch shape
    predictions = []
    for i in range(0,image_test.shape[0],test_batch):
        if i%90000==0:
            print(i)
        batch_images = image_test[i:i+test_batch]
        batch_pred = sess.run(tf.argmax(output, 1), feed_dict = {final_input: batch_images,is_training:False})
        for j in range(test_batch):
            predictions.append(batch_pred[j])

In [ ]:
test_classes = np.asarray([labels_unique[predictions[i]] for i in range(len(predictions))])
test_df = pd.DataFrame(test_classes)
test_df['id'] = pd.Series([i for i in range(1,len(predictions)+1)])
test_df['label'] = test_df[test_df.columns[0]]
test_df = test_df[['id','label']]
test_df

In [57]:
test_df.to_csv('cnn_week2_temp.csv',index=False)